<div style="text-align: center;"> <img align=middle src="https://hyperpix.net/wp-content/uploads/2020/04/ford-v-ferrari-logo-font-free-download-856x484.jpg"> </div>

# Проект 7. "Ford vs Ferrari: определяем модель авто по фото"
### Выполнен Шашановым М. (SF-DST-76)

**Задачей данного проекта является построение модели классификации автомобилей по их фотографиям, всего 10 различных моделей.
<br>
Основная идея решения: взять предобученную на ImageNet нейронную сеть EfficientNet и дообучить под нашу задачу, т.е. применить Transfer Learning. Дообучение необходимо, поскольку при обучении исходной сети не использовались изображения этих моделей автомобилей. Плюсом данного метода является то, что обучение нейронной сети с нуля требует большого числа примеров изображений и долгого времени вычислений, что не подходит для выполнения данного проекта.**

In [1]:
# Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import pickle
import zipfile
import csv
import cv2
import sys
import os
import random
import time
import gc
import string
import pathlib
import itertools
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

from tensorflow.keras import Sequential
from tensorflow.keras.activations import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.optimizers.schedules import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

import car_class_kernel_module as km

import PIL
from PIL import ImageOps, ImageFilter, Image

import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('WARNING')

In [2]:
# Установим дефолтный размер графиков и подписей
from pylab import rcParams

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['font.size'] = 12
plt.rcParams['legend.fontsize'] = 'large'
plt.rcParams['figure.titlesize'] = 'medium'

# Графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [3]:
!pip freeze > requirements.txt

In [4]:
# Проверим версии основных библиотек
print('Python       :', sys.version.split('\n')[0].split(' | ')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)
print('PIL          :', PIL.__version__)

Работаем с Tensorflow v2

In [5]:
# Проверка GPU
!nvidia-smi -L
tf.test.gpu_device_name()

In [6]:
# Очищаем сессию Tensorflow
K.clear_session()

# Загрузка и исследование данных

Загрузим и проведем разведывательный анализ (EDA) исходных изображений. В данном проекте (соревнованияи Kaggle) предоставляются размеченные обучающие данные и неразмеченные тестовые в архивированном виде.

In [7]:
# Исходные данные и директории
print(os.listdir('..'))
print(os.listdir('../input/'))
print(os.listdir('../input/sf-dl-car-classification/'))

In [8]:
DATA_PATH = '../input/sf-dl-car-classification/' # директория с исходными данными
PATH = '../working/car/' # рабочая директория для распакованных изображений
MODELS_PATH = '../working/saved_models/' # директория для сохранения состояния модели после разных эпох обучения

In [9]:
# Создание директорий
os.makedirs(MODELS_PATH, exist_ok=True, mode=0o777)
print(os.path.exists(MODELS_PATH))
os.makedirs(PATH, exist_ok=True, mode=0o777)
print(os.path.exists(PATH))

In [10]:
# Распаковываем картинки
# %%time
# # Will unzip the files so that you can see them
# for data_zip in ['train.zip', 'test.zip']:
#     with zipfile.ZipFile(DATA_PATH+data_zip,"r") as z:
#         z.extractall(PATH)
# print(os.listdir(PATH))

In [11]:
# Распаковываем картинки
%%time
!unzip -q {DATA_PATH + 'train.zip'} -d {PATH}
!unzip -q {DATA_PATH + 'test.zip'} -d {PATH}
print(os.listdir(PATH))

In [12]:
# Пути к изображениям для обучения и предсказания
train_path = PATH + 'train/'
sub_path = PATH + 'test_upload/'

In [13]:
# Загрузка табличных данных
train_df = pd.read_csv(DATA_PATH+'train.csv')
submission_df = pd.read_csv(DATA_PATH+'sample-submission.csv')
train_df_init = train_df.copy()

В train_df (таблица с изображениями для обучения) столбцы Id и Category, пропусков нет

In [14]:
data = train_df
display(data.info(), data.isna().sum(), data.head())

В submission_df (таблица с изображениями для предсказания) столбцы Id и Category, пропусков нет

In [15]:
data = submission_df
display(data.info(), data.isna().sum(), data.head())

Выведем информацию об изображениях: расположение по директориям и пр.

In [16]:
train_filenames_list = []
train_categories_list = []
train_list = os.listdir(train_path)
test_list = os.listdir(sub_path)
print(len(train_list), len(test_list), train_df.shape[0])
print(train_list)
train_count = 0
for dir_path in train_list:
    train_filenames = os.listdir(train_path + dir_path)
    train_count += len(train_filenames)
    train_filenames_list += train_filenames
    train_categories_list += [int(dir_path)] * len(train_filenames)
    print(train_filenames[:5])

print(train_count == train_df.shape[0])
print(test_list[:10])

Далее проверим, что изображения в директории "train/" соответствуют данным в датафрейме train_df

In [17]:
train_df_test = pd.DataFrame({
    'Id' : train_filenames_list,
    'Category' : train_categories_list,
}, columns=['Id', 'Category'])

train_df.sort_values(by=['Id'], inplace=True, ignore_index=True)
train_df_test.sort_values(by=['Id'], inplace=True, ignore_index=True)
train_df.equals(train_df_test)

Добавим и проанализируем дополнительные признаки изображений, такие, как формат, размер и пр.

In [18]:
train_df = train_df_init
train_df['Path'] = train_df.apply(lambda row: 'train/' + str(row['Category']) + '/' + row['Id'], axis=1)

In [19]:
def add_image_param_columns(df):
    df['Format'] = df['Path'].apply(lambda img_path: PIL.Image.open(PATH+img_path).format)
    df['Mode'] = df['Path'].apply(lambda img_path: PIL.Image.open(PATH+img_path).mode)
    df['Width'] = df['Path'].apply(lambda img_path: PIL.Image.open(PATH+img_path).width)
    df['Height'] = df['Path'].apply(lambda img_path: PIL.Image.open(PATH+img_path).height)
    df['Size'] = df['Path'].apply(lambda img_path: os.path.getsize(PATH+img_path))
    df['HeightDivWidth'] = df['Height'].values / df['Width'].values
    df['NumPixels'] = df['Height'].values * df['Width'].values

In [20]:
add_image_param_columns(train_df)
display(train_df.head())

In [21]:
submission_df.Category = 0
submission_df['Path'] = submission_df.apply(lambda row: 'test_upload/' + row['Id'], axis=1)
add_image_param_columns(submission_df)
display(submission_df.head())

Пример изображения из обучающей выборки, размер 640 пикселей по ширине и 480 по высоте

In [22]:
img_path = PATH + train_df['Path'].values[0]
img = PIL.Image.open(img_path)
plt.imshow(img)

## Format

In [23]:
col = 'Format'
km.print_col_info(train_df[col])
km.print_col_info(submission_df[col])

Все изображения в формате JPEG

## Mode

In [24]:
col = 'Mode'
km.print_col_info(train_df[col], 2)
km.print_col_info(submission_df[col], 2)

In [25]:
display(train_df[col].value_counts())
display(submission_df[col].value_counts())

Среди изображений встречается несколько черно-белых (L), остальные цветные (RGB).
<br>
Посмотрим подробнее на черно-белые:

In [26]:
train_df_black_and_white = train_df[train_df['Mode'].str.contains('L')]
display(train_df_black_and_white)

In [27]:
submission_df_black_and_white = submission_df[submission_df['Mode'].str.contains('L')]
display(submission_df_black_and_white)

In [28]:
img_path = PATH + submission_df_black_and_white['Path'].values[1]
img = PIL.Image.open(img_path)
plt.imshow(img)

Удаление или конвертация таких изображений в RGB не улучшает качества предсказания, поэтому оставим их как есть в обучающей и тестовых выборках

## Width

In [29]:
col = 'Width'
km.print_col_info(train_df[col])
km.print_col_info(submission_df[col])

In [30]:
print([train_df[col].min(), train_df[col].max(), round(train_df[col].mean())])
print([submission_df[col].min(), submission_df[col].max(), round(submission_df[col].mean())])

In [31]:
km.plot_num_col_unified(train_df, submission_df, col)

Большинство изображений имеет максимальную ширину, равную 640 пикселей (в среднем 610)

## Height

In [32]:
col = 'Height'
km.print_col_info(train_df[col])
km.print_col_info(submission_df[col])

In [33]:
print([train_df[col].min(), train_df[col].max(), round(train_df[col].mean())])
print([submission_df[col].min(), submission_df[col].max(), round(submission_df[col].mean())])

In [34]:
km.plot_num_col_unified(train_df, submission_df, col)

Большая часть изображений имеет высоту 480 пикселей, в среднем 440. Также довольно высокий процент изображений имеют высоту 360 пикселей

## Size

In [35]:
col = 'Size'
km.print_col_info(train_df[col], 3)
km.print_col_info(submission_df[col], 3)

In [36]:
print([train_df[col].min(), train_df[col].max(), round(train_df[col].mean())])
print([submission_df[col].min(), submission_df[col].max(), round(submission_df[col].mean())])

In [37]:
km.plot_num_col_unified(train_df, submission_df, col)

Размер изображений имеет нормальное распределение как в обучающей выборке, так и в тестовой

## Category

In [38]:
col = 'Category'
km.print_col_info(train_df[col], 1, 10)

In [39]:
km.plot_classes_hist(train_df[col])

Распределение классов в обучающей выборке достаточно равномерное, для обучения это хорошо

In [40]:
def get_num_files(path):
    tree = os.walk(path)
    num_files = 0
    for dir in tree:
        num_files += len(dir[2])
    return num_files


def delete_files(filenames, dir=PATH):
    for filename in filenames:
        os.remove(dir+filename)

In [41]:
# n1 = get_num_files(train_path)
# delete_files(train_df_black_and_white['Path'].values.tolist())
# n2 = get_num_files(train_path)
# print(n1 - n2)

# Основные настройки

In [42]:
# Установим основные настройки
EPOCHS               = 15  # Эпохи обучения
BATCH_SIZE           = 32 # уменьшаем batch если сеть большая, иначе не помещается в память на GPU
LR                   = 1e-3 # Learning Rate
VAL_SPLIT            = 0.2 # Сколько данных выделяем на тест = 20%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = (160, 215) # Какого размера подаем изображения в сеть, для отладки (90, 120), (135, 180)
IMG_CHANNELS         = 3 # У RGB 3 канала
INPUT_SHAPE          = (*IMG_SIZE, IMG_CHANNELS)

# Устаналиваем конкретное значение random seed для воспроизводимости
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
PYTHONHASHSEED = RANDOM_SEED
tf.random.set_seed(RANDOM_SEED)

# Подготовка данных

In [43]:
# Названия моделей автомобилей
class_names = [
  'Приора', #0
  'Ford Focus', #1
  'Самара', #2
  'ВАЗ-2110', #3
  'Жигули', #4
  'Нива', #5
  'Калина', #6
  'ВАЗ-2109', #7
  'Volkswagen Passat', #8
  'ВАЗ-21099' #9
]
class_names_dict = dict(zip(range(CLASS_NUM), class_names))
# pprint(class_names_dict)

Создаем генераторы изображений, т.е. объекты ImageDataGenerator, указав какие аугментации (преобразования изображений) необходимо выполнить.
Далее при обучении мы каждый раз "показываем" нейронной сети немного разные варианты одной и той же картинки. Обучение с аугментациями данных - это один из способов регуляризации (повышения обобщающей способности модели), особенно при работе с небольшим датасетом для обучения.
<br>
Используются следующие виды аугментаций:
* horizontal_flip, отражение по горизонтали с вероятностью 50%
* rotation_range, случайный поворот в пределах 10 градусов
* shear_range, случайный сдвиг с искажением
* brightness_range, случайное изменение яркости

Валидационные и тестовые изображения оставим без аугментации. 
<br>
Затем укажем, откуда брать изображения. При этом применим интерполяцию изображений типа 'hamming' для улучшения качества обучения и последующего предсказания.

In [44]:
image_interpolation = 'hamming'

train_datagen = ImageDataGenerator(
#     rescale=1/255,
    validation_split=VAL_SPLIT,

    # ниже параметры аугментаций:
    horizontal_flip=True,
    rotation_range=10,
    shear_range=0.2,
    brightness_range=(0.8, 1.2),
)

val_datagen = ImageDataGenerator(
#     rescale=1/255,
    validation_split=VAL_SPLIT,
)

sub_datagen = ImageDataGenerator(
#     rescale=1/255
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    interpolation=image_interpolation,
    shuffle=True,
    seed=RANDOM_SEED,
    subset='training'
)

val_generator = val_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    interpolation=image_interpolation,
    shuffle=True,
    seed=RANDOM_SEED,
    subset='validation'
)

sub_generator = sub_datagen.flow_from_dataframe( 
    dataframe=submission_df,
    directory=sub_path,
    x_col='Id',
    y_col=None,
    class_mode=None,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    interpolation=image_interpolation,
    shuffle=False
)

Для примера первые несколько изображений каждого генератора

In [45]:
print('Train:')
km.show_first_images(train_generator)

print('Val:')
km.show_first_images(val_generator)

print('Sub:')
km.show_first_images(sub_generator, labels=False)

# Модель и ее обучение

Загружаем предобученную сеть EfficientNetB7, одну из наиболее эффективных на данный момент, отключая голову модели. В данной модели 813 слоев.


In [46]:
# Предобученная нейросеть EfficientNetB7 из модуля keras.applications
base_model = EfficientNetB7(weights='imagenet',
                            include_top=False,
                            input_shape=INPUT_SHAPE)

print(base_model.trainable)
print(f"Num layers: {len(base_model.layers)}")

Устанавливаем новую «голову» (head):



In [47]:
# Строим модель
model = Sequential([
  base_model, 
  GlobalMaxPool2D(),
  Dropout(0.5),
  Dense(10)
])

Компилируем модель, используя ExponentialDecay в качестве техники управления Learning Rate. Включение параметра amsgrad немного улучшает результат предсказания.

In [48]:
model.compile(loss=CategoricalCrossentropy(from_logits=True),
              optimizer=Adam(ExponentialDecay(LR, 100, 0.9), amsgrad=True),
              metrics='accuracy')

После каждой эпохи обучения добавим некоторые действия (коллбэки):
* ModelCheckpoint сохраняет модель в файл best_model.h5 (в директории MODELS_PATH) в том случае, если значение val_accuracy после текущей эпохи достигло максимума. Далее эти веса модели можно подгрузить для финального предсказания.
* TerminateOnNaN прекращает обучение, если loss стал равным NaN.
* LambdaCallback позволяет задать действия после конца эпохи: сохранить веса модели и фактический Learning Rate

In [49]:
checkpoint = ModelCheckpoint(MODELS_PATH+'best_model.h5' , monitor='val_accuracy', verbose=1  , mode='max', save_best_only=True)
terminate = TerminateOnNaN()

def save_weights_after_each_epoch(epoch, logs):
    model_name = 'model_epoch_' + str(epoch) + '.h5'
    model.save_weights(MODELS_PATH+model_name)

each_epoch_1 = LambdaCallback(on_epoch_end=save_weights_after_each_epoch)

model_lr_list = []

def get_lr_after_each_epoch(epoch, logs):
    lr = K.eval(model.optimizer._decayed_lr(tf.float32))
    model_lr_list.append(lr)
    print(f"LR: {lr:.2e}\n")
    
each_epoch_2 = LambdaCallback(on_epoch_end=get_lr_after_each_epoch)

callbacks_list = [checkpoint, terminate, each_epoch_1, each_epoch_2]

In [ ]:
# Сбросим генераторы на начало перед процессом обучения
train_generator.reset()
val_generator.reset()

In [51]:
lr = K.eval(model.optimizer._decayed_lr(tf.float32))
model_lr_list.append(lr)
print(f"Начальный Learning Rate:: {lr:.2e}\n")

In [52]:
# Обучаем модель
%%time
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=EPOCHS,
                    callbacks=callbacks_list)

В процессе обучения происходит плавное уменьшение Learning Rate от шага к шагу, используя ExponentialDecay
(экспоненциальное затухание). Параметры затухания подбирались с целью улучшения точности предсказания

In [53]:
km.plot_learning_rate(model_lr_list, EPOCHS)

Посмотрим на графики обучения. Заметно, что точность на валидации стагнировала на последних эпохах, что похоже на переобучение,

In [54]:
km.plot_history(history)

In [55]:
# Сохраненные модели
display(sorted(os.listdir(MODELS_PATH)))

# Предсказание на примере

Получить предсказание обученной модели для произвольного изображения можно таким образом:

In [56]:
# Загружаем лучшую по точности модель
model.load_weights(MODELS_PATH+'best_model.h5')
model.compile(loss=CategoricalCrossentropy(from_logits=True),
              optimizer=Adam(ExponentialDecay(LR, 100, 0.9), amsgrad=True),
              metrics='accuracy')

In [57]:
# Скачиваем изображение по URL, это модель Форд Фокус
!wget -q https://www.masmotors.ru/colors/ford-focus/1.png -P ../working/car/

sample_filename = '../working/car/1.png'
print(os.path.exists(sample_filename))    

#какого размера изображение модель принимает на вход?
img_size = np.array(model.input.shape)[[2, 1]]
print(img_size)

# загружаем изображение с помощью cv2
image = cv2.imread(sample_filename, cv2.IMREAD_COLOR)[..., ::-1] #cv2.imread читает в формате BGR, конвертируем в RGB с помощью индекса ::-1
image = cv2.resize(image, img_size)

image = np.array(Image.open(sample_filename).convert('RGB').resize(img_size)) #.convert('RGB') нужен на случай, если изображение черно-белое

# Изображение
plt.imshow(image)
plt.show()

In [58]:
# превращаем изображение в батч из одного изображения, добавляя новую ось в начале
image = image[None, ...]

# получаем батч предсказаний и берем нулевой элемент
pred = model.predict(image)[0]

# берем индекс класса с максимальным значением
class_idx = pred.argmax()

# получаем название
print(class_idx)
print(class_names_dict[class_idx])

Предсказание модели автомобиля по тестовому изображению из Интернета правильное

# Test-time augmentations (TTA)

Этот термин означает применение аугментаций к изображениям при инференсе для улучшения качества предсказаний. Для каждого изображения мы получаем несколько предсказаний и усредняем их.

In [59]:
NUM_TTA_STEPS = 3 # количество предсказаний для усреднения

In [60]:
# Создадим отдельные обьекты генераторов с аугментациями
sub_tta_datagen = ImageDataGenerator(
#     rescale=1/255,
    # ниже параметры аугментаций:
    horizontal_flip=True,
    rotation_range=10,
    shear_range=0.2,
    brightness_range=(0.8, 1.2),
)

sub_tta_generator = sub_tta_datagen.flow_from_dataframe(
    dataframe=submission_df,
    directory=sub_path,
    x_col='Id',
    y_col=None,
    class_mode=None,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    interpolation=image_interpolation,
    shuffle=False
)

# Предсказание на тестовых данных

Добавим функции для получения и сохранения предсказаний

In [61]:
predictions_dict = {}

In [62]:
def get_predictions(model, apply_tta=True, predictions_tta=None):
    if not apply_tta:
        sub_generator.reset()
        predictions = model.predict(sub_generator, verbose=1)
        return predictions
    else:
        predictions_list = []
        for _ in range(NUM_TTA_STEPS):
            sub_tta_generator.reset()
            predictions_list.append(model.predict(sub_tta_generator, verbose=1))
        predictions_arr = np.array(predictions_list)
        predictions = predictions_arr.mean(axis=0)
        if not predictions_tta:
            return predictions
        else:
            return predictions, predictions_arr

In [63]:
def save_predictions_to_csv(predictions, filename):
    predictions = predictions.argmax(axis=-1)
    df = pd.DataFrame({
        'Id': sub_generator.filenames,
        'Category': predictions
        }, columns=['Id', 'Category'])
    df.to_csv(filename+'.csv', index=False)
    if not np.array_equal(df.Id.values, submission_df.Id.values):
        print('Submission results have incorrect order')
    if predictions.shape[0] != submission_df.shape[0]:
        print('Submission results have incorrect amount')

Далее подгрузим лучшую итерацию в обучении (best_model):

In [64]:
model.load_weights(MODELS_PATH+'best_model.h5')
model.compile(loss=CategoricalCrossentropy(from_logits=True),
              optimizer=Adam(ExponentialDecay(LR, 100, 0.9), amsgrad=True),
              metrics='accuracy')

Получим предсказания на тестовых данных без применения TTA

In [65]:
%%time
predictions = get_predictions(model, False)
predictions_dict['best_wo_tta'] = predictions
print(predictions.shape)
print(predictions.argmax(axis=-1).shape)
km.plot_classes_hist(predictions.argmax(axis=-1))

Далее для той же лучшей модели получим предсказание с применением TTA

In [66]:
%%time
predictions, predictions_arr = get_predictions(model, True, True)
predictions_dict['best_tta'] = predictions
print(predictions.shape)
print(predictions.argmax(axis=-1).shape)
km.plot_classes_hist(predictions.argmax(axis=-1))

Сделаем предсказания 3 раза. Нулевая ось - номер попытки, первая ось - номер изображения, вторая ось - номер класса. Теперь нам нужно сделать усреднение по номеру попытки, а затем argmax по номеру класса. 
<br>
Также можем посмотреть насколько совпали предсказания на каждом изображении с разными аугментациями (цвет на изображении означает класс):

In [67]:
plt.figure(figsize=(15, 3))
plt.imshow(predictions_arr.argmax(axis=-1)[:, :50], cmap='nipy_spectral')
plt.xlabel('Номер изображения')
plt.ylabel('Номер попытки')
plt.show()

Далее подгрузим модель с последней эпохи обучения (не обязательно лучшую), и сделаем предсказания на тестовых данных применяя TTA

In [68]:
model_name = 'model_epoch_' + str(EPOCHS-1) + '.h5'
print(model_name)
model.load_weights(MODELS_PATH+model_name)
model.compile(loss=CategoricalCrossentropy(from_logits=True),
              optimizer=Adam(ExponentialDecay(LR, 100, 0.9), amsgrad=True),
              metrics='accuracy')

In [69]:
%%time
predictions, predictions_arr = get_predictions(model, True, True)
predictions_dict['last_tta'] = predictions
print(predictions.shape)
print(predictions.argmax(axis=-1).shape)
km.plot_classes_hist(predictions.argmax(axis=-1))

Далее применим простой способ ансамблирования - усреднение предсказаний нескольких моделей (например, лучшей и последней).
<br>
Скорее всего таким образом удасться увеличить точность предсказаний, но не обязательно

In [77]:
predictions = np.array([predictions_dict['best_tta'], predictions_dict['last_tta']])
predictions_average = predictions.mean(axis=0)
predictions_dict['average_tta'] = predictions_average
print(predictions.shape)
print(predictions_average.shape)
print(predictions_average.argmax(axis=-1).shape)
km.plot_classes_hist(predictions_average.argmax(axis=-1))

In [71]:
# print(gc.isenabled())
# del base_model
# del model
# gc.collect()
# # скидываем сессию
# K.clear_session()

In [72]:
# Очистим рабочие директории, чтобы не помешали сохранить CSV файлы с предсказаниями
km.clear_directory(PATH)
km.clear_directory(MODELS_PATH)
print(os.listdir('../working/'))

Сохраним несколько вариантов предсказаний для submission, чтобы выбрать лучший из них

In [78]:
for filename, predictions in predictions_dict.items():
    save_predictions_to_csv(predictions, 'submission_'+filename)